In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.svm import SVR
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
from tqdm import tqdm
import sys
from pvlib import solarposition
import sys
import os
sys.path.append(os.path.abspath('../common/'))
import common_functions_by_date as cfbd
import datetime
import predictions_algorithmes as pa

In [2]:
weather_path = 'Input/weather_train_set3.csv'
demand_path = 'Input/demand_train_set3.csv'
solar_path = 'Input/pv_train_set3.csv'
dp = cfbd.DataPreprocesser(weather_path, demand_path, solar_path)
dp.load_df()
dp.remove_nan()
dp.interpolate_df()
dp.get_zenith_angle()
dp.get_poa_and_ghi_irradiance()
dp.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location5,solar_location1,week,dow,date,hour,sp,zenith_angle,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-03 00:00:00,2.19,0.0,0.0,7.05,7.46,13.20,9.640,6.680,13.090,8.560,...,0.0,0.0,44,4,2017-11-03,0,1.0,122.821452,0.0,0.0
2017-11-03 00:30:00,2.14,0.0,0.0,7.38,7.30,13.26,9.675,6.475,13.150,8.625,...,0.0,0.0,44,4,2017-11-03,0,2.0,115.780705,0.0,0.0
2017-11-03 01:00:00,2.01,0.0,0.0,7.70,7.14,13.32,9.710,6.270,13.210,8.690,...,0.0,0.0,44,4,2017-11-03,1,3.0,108.668070,0.0,0.0
2017-11-03 01:30:00,1.87,0.0,0.0,7.48,7.00,13.34,9.720,6.090,13.255,8.715,...,0.0,0.0,44,4,2017-11-03,1,4.0,101.505357,0.0,0.0
2017-11-03 02:00:00,1.86,0.0,0.0,7.20,6.86,13.36,9.730,5.910,13.300,8.740,...,0.0,0.0,44,4,2017-11-03,2,5.0,94.307845,0.0,0.0


In [3]:
dp.df.tail()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location5,solar_location1,week,dow,date,hour,sp,zenith_angle,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2019-12-17 21:30:00,3.50,0.0,0.0,1.53,3.255,9.500,5.040,4.065,9.48,4.810,...,0.0,0.0,51,1,2019-12-17,21,44.0,149.475764,0.0,0.0
2019-12-17 22:00:00,3.21,0.0,0.0,2.75,2.970,9.520,4.770,4.020,9.49,4.470,...,0.0,0.0,51,1,2019-12-17,22,45.0,145.619973,0.0,0.0
2019-12-17 22:30:00,2.83,0.0,0.0,3.16,2.745,9.595,4.695,3.850,9.54,4.355,...,0.0,0.0,51,1,2019-12-17,22,46.0,140.770523,0.0,0.0
2019-12-17 23:00:00,2.55,0.0,0.0,1.52,2.520,9.670,4.620,3.680,9.59,4.240,...,0.0,0.0,51,1,2019-12-17,23,47.0,135.251810,0.0,0.0
2019-12-17 23:30:00,2.32,0.0,0.0,2.70,2.520,9.670,4.620,3.680,9.59,4.240,...,0.0,0.0,51,1,2019-12-17,23,48.0,129.284892,0.0,0.0


In [4]:
first_day_pred=datetime.datetime(2019,12,18).date()

In [5]:
mp = cfbd.MLPredictor(dp,first_day_pred)
mp.get_demand_previous_week()
mp.get_weather_prediction(weather_path)
mp.pred_demand_with_forecast_method_and_average_with_previous_weeks(dp.df, 'Output/rectify_forecast', first_day_pred, 'demand_MW', compute_forecast=True)
mp.predict_pv_power_smooth_and_square_irr()

week prediction with start day :  2019-12-18
  0%|          | 0/336 [00:00<?, ?it/s]2019-12-18 00:00:00
KNeighborsRegressor(n_neighbors=59)
100%|██████████| 336/336 [01:01<00:00,  5.43it/s]
week prediction with start day :  2019-12-11
  0%|          | 0/336 [00:00<?, ?it/s]2019-12-11 00:00:00
KNeighborsRegressor(n_neighbors=59)
100%|██████████| 336/336 [01:01<00:00,  5.49it/s]
week prediction with start day :  2019-12-04
  0%|          | 0/336 [00:00<?, ?it/s]2019-12-04 00:00:00
KNeighborsRegressor(n_neighbors=59)
100%|██████████| 336/336 [01:00<00:00,  5.55it/s]
week prediction with start day :  2019-11-27
  0%|          | 0/336 [00:00<?, ?it/s]2019-11-27 00:00:00
KNeighborsRegressor(n_neighbors=53)
100%|██████████| 336/336 [01:00<00:00,  5.59it/s]
week prediction with start day :  2019-11-20
  0%|          | 0/336 [00:00<?, ?it/s]2019-11-20 00:00:00
KNeighborsRegressor(n_neighbors=50)
100%|██████████| 336/336 [00:58<00:00,  5.73it/s]
smooth_phaze : -3
weather dephasage : 0
smooth_pha

In [6]:
bpd = cfbd.BatteryPowerDispatcher
B = bpd.get_all_dispatch_in_a_week(bpd,mp.predicted_df, first_day_pred)
B_final = bpd.format_dispatching_for_competition(B, mp.predicted_df.index)

In [7]:
B.sum()

20191218    0.000000e+00
20191219    6.661338e-16
20191220   -6.661338e-16
20191221    1.776357e-15
20191222    1.332268e-15
20191223   -2.442491e-15
20191224    1.554312e-15
dtype: float64

In [8]:
B_final.to_csv('Output/BGBattery_last_model_set3.csv')

## Naive prediction

In [9]:
mp.get_demand_previous_week()
mp.get_solar_power_previous_week()

,week,dow,sp,hour,zenith_angle,GHI,POA,demand_MW,temp_location3,temp_location6,...,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,pv_power_mw
datetime,,,,,,,,,,,,,,,,,,,,,
2019-12-18 00:00:00,51.0,2.0,1.0,0.0,123.013989,0.0,0.0,2.45,1.980,9.770,...,3.360,9.690,4.040,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-18 00:30:00,51.0,2.0,2.0,0.0,116.533359,0.0,0.0,2.40,1.855,9.805,...,3.220,9.740,3.980,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-18 01:00:00,51.0,2.0,3.0,1.0,109.906089,0.0,0.0,2.24,1.730,9.840,...,3.080,9.790,3.920,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-18 01:30:00,51.0,2.0,4.0,1.0,103.175952,0.0,0.0,2.16,1.640,9.900,...,2.830,9.875,3.965,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-18 02:00:00,51.0,2.0,5.0,2.0,96.374762,0.0,0.0,2.13,1.550,9.960,...,2.580,9.960,4.010,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24 21:30:00,52.0,1.0,44.0,21.0,149.801253,0.0,0.0,3.50,6.470,9.590,...,5.695,9.560,7.290,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-24 22:00:00,52.0,1.0,45.0,22.0,146.092289,0.0,0.0,3.21,6.370,9.410,...,5.540,9.460,7.230,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-12-24 22:30:00,52.0,1.0,46.0,22.0,141.344982,0.0,0.0,2.83,6.150,9.330,...,5.535,9.400,7.045,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
bpd = cfbd.BatteryPowerDispatcher
B = bpd.get_all_dispatch_in_a_week(bpd,mp.predicted_df, first_day_pred)
B_final = bpd.format_dispatching_for_competition(B, mp.predicted_df.index)

In [11]:
B_final.to_csv('Output/BGBattery_naive_set3.csv')

## Dispatch with real values

In [12]:
weather_path_task4 = '../task4/Input/weather_train_set4.csv'
demand_path_task4 = '../task4/Input/demand_train_set4.csv'
solar_path_task4 = '../task4/Input/pv_train_set4.csv'
dp_task4 = cfbd.DataPreprocesser(weather_path_task4, demand_path_task4, solar_path_task4)
dp_task4.load_df()
dp_task4.remove_nan()
dp_task4.interpolate_df()
# dp.get_zenith_angle()
dp_task4.get_poa_and_ghi_irradiance()
dp_task4.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location4,solar_location5,solar_location1,week,dow,date,hour,sp,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-03 00:00:00,2.19,0.0,0.0,7.05,7.46,13.20,9.640,6.680,13.090,8.560,...,0.0,0.0,0.0,44,4,2017-11-03,0,1.0,0.0,0.0
2017-11-03 00:30:00,2.14,0.0,0.0,7.38,7.30,13.26,9.675,6.475,13.150,8.625,...,0.0,0.0,0.0,44,4,2017-11-03,0,2.0,0.0,0.0
2017-11-03 01:00:00,2.01,0.0,0.0,7.70,7.14,13.32,9.710,6.270,13.210,8.690,...,0.0,0.0,0.0,44,4,2017-11-03,1,3.0,0.0,0.0
2017-11-03 01:30:00,1.87,0.0,0.0,7.48,7.00,13.34,9.720,6.090,13.255,8.715,...,0.0,0.0,0.0,44,4,2017-11-03,1,4.0,0.0,0.0
2017-11-03 02:00:00,1.86,0.0,0.0,7.20,6.86,13.36,9.730,5.910,13.300,8.740,...,0.0,0.0,0.0,44,4,2017-11-03,2,5.0,0.0,0.0


In [13]:
real_value_df = dp_task4.df[(dp_task4.df.index.date >= first_day_pred) & (dp_task4.df.index.date < first_day_pred + datetime.timedelta(days=7))]

In [14]:
bpd = cfbd.BatteryPowerDispatcher
B_real = bpd.get_all_dispatch_in_a_week(bpd,real_value_df, first_day_pred)
B_final_real = bpd.format_dispatching_for_competition(B_real, real_value_df.index)

In [15]:
B_final_real.to_csv('Output/BGBattery_real_set3.csv')